In [ ]:
import torch
import mxnet as mx
import numpy as np
import cv2
model = torch.hub.load('pytorch/vision:v0.4.2', 'resnet50', pretrained=True)

In [ ]:
import numpy as np
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch
import matplotlib
import matplotlib.pyplot as plt


def Action(image_Tensor, cord, action, increment):
    cord_new = cord.clone()

    # image_Tensor = Image tensor[3,224,224]

    # cord = [xl,xh,yl,yh]

    # action = 0: increase xl by increment
    # action = 1: decrease xl by increment
    # action = 2: increase xh by increment
    # action = 3: decrease xh by increment
    # action = 4: increase yl by increment
    # action = 5: decrease yl by increment
    # action = 6: increase yl by increment
    # action = 7: decrease yl by increment
    # action = 8: Trigger

    # increment = movmenet value

    pil2tensor = transforms.ToTensor()
    tensor2pil = transforms.ToPILImage()

    transform = transforms.Compose(
        [transforms.Resize([224, 224])])

    if action == 0:
        cord_new[0] += increment
    if action == 1:
        cord_new[0] -= increment
    if action == 2:
        cord_new[1] += increment
    if action == 3:
        cord_new[1] -= increment
    if action == 4:
        cord_new[2] += increment
    if action == 5:
        cord_new[2] -= increment
    if action == 6:
        cord_new[3] += increment
    if action == 7:
        cord_new[3] -= increment

    cord_new[0] = np.minimum(222, cord_new[0])
    cord_new[1] = np.minimum(223, cord_new[1])
    if cord_new[0] >= cord_new[1]:
        cord_new[0] = cord_new[1] - 1
    cord_new[2] = np.minimum(222, cord_new[2])
    cord_new[3] = np.minimum(223, cord_new[3])
    if cord_new[2] >= cord_new[3]:
        cord_new[2] = cord_new[3] - 1

    Img_cut = image_Tensor[:, cord_new[0]:cord_new[1], cord_new[2]:cord_new[3]]
    Img_cut=Img_cut.cpu()
    plt.figure()
    plt.imshow(Img_cut.numpy().transpose(1, 2, 0))
    plt.show()
    print(image_Tensor.type())
    Img_PIL =transforms.functional.to_pil_image(image_Tensor.cpu(), mode="RGB")
    plt.figure()
    print(Img_PIL)
    plt.imshow(Img_PIL)
    plt.show()
    Img_PIL = transform(Img_PIL)

    Img_new = pil2tensor(Img_PIL)
    print(Img_new)

    return Img_new, cord_new

In [ ]:
import math

import torch
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F

# Based on Resnet implementation from: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


# These layers are based on DetNet: A Backbone network for Object Detection, https://arxiv.org/pdf/1804.06215.pdf
class DetnetBottleneck(nn.Module):
    # We keep the same grid size in the output. (SxS)
    # Layer structre is 1x1 conv, dilated 3x3 conv, 1x1 conv, with a skip connection
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, block_type='A'):
        super(DetnetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=2, bias=False, dilation=2)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.downsample = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes or block_type == 'B':
            self.downsample = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.downsample(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, layers):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.layer6=nn.Linear(100352,2048)
        self.layer7=nn.Linear(2048,1024)
        self.layer8=nn.Linear(1024,20)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = [block(self.inplanes, planes, stride, downsample)]
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def _make_detnet_layer(self,in_channels):
        layers = []
        layers.append(DetnetBottleneck(in_planes=in_channels, planes=256, block_type='B'))
        layers.append(DetnetBottleneck(in_planes=256, planes=256, block_type='A'))
        layers.append(DetnetBottleneck(in_planes=256, planes=256, block_type='A'))
        return nn.Sequential(*layers)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)        
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(batch_size, -1) 
        x = self.layer6(x)
        x = self.relu(x)
        features = self.layer7(x)
        x=self.relu(features)
        x=self.layer8(x)
        return torch.nn.functional.softmax(x[0],dim=0),features


def update_state_dict(pretrained_model, model):
    new_state_dict = pretrained_model.state_dict()
    dd = model.state_dict()
    for k in new_state_dict.keys():
        if k in dd.keys() and not k.startswith('fc'):
            dd[k] = new_state_dict[k]
    model.load_state_dict(dd)
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        pretrained_model = models.resnet50(pretrained=True)
        model = update_state_dict(pretrained_model, model)
    return model



In [ ]:
import numpy as np
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch
import matplotlib
import matplotlib.pyplot as plt
import torch.nn.functional as F
import math


def Action(image_Tensor, cord, action, increment):
    cord_new = cord.clone().detach()
    # image_Tensor = Image tensor[3,224,224]
    special_increment=int(math.sqrt((increment*increment)/2))
    # cord = [x1,y1,x2,y2]



    # increment = movmenet value

    pil2tensor = transforms.ToTensor()
    tensor2pil = transforms.ToPILImage()
    action_type=""
    transform = transforms.Compose(
        [transforms.Resize([224, 224])])

    if action == 0: # move the box right
        action_type="move right"
        cord_new[0] += increment
        cord_new[2] += increment
    if action == 1: # move the box left
        action_type="move left"        
        cord_new[0] -= increment
        cord_new[2] -= increment
    if action == 2: # move the box up
        action_type="move up"
        cord_new[1] += increment
        cord_new[3] += increment
    if action == 3: # move the box down
        action_type="move down"
        cord_new[1] -= increment
        cord_new[3] -= increment
    if action == 4: # zoom in
        action_type="zoom in"
        cord_new[0] += special_increment
        cord_new[1] += special_increment
        cord_new[2] -= special_increment
        cord_new[3] -= special_increment
    if action == 5: # zoom out
        action_type="zoom out"
        cord_new[0] -= special_increment
        cord_new[1] -= special_increment
        cord_new[2] += special_increment
        cord_new[3] += special_increment
        
    cord_new[0] = np.maximum(0, cord_new[0])
    cord_new[1] = np.maximum(0, cord_new[1])
    cord_new[2] = np.maximum(0, cord_new[2])
    cord_new[3] = np.maximum(0, cord_new[3])
        
    cord_new[0] = np.minimum(223, cord_new[0])
    cord_new[2] = np.minimum(223, cord_new[1])
    if cord_new[0] >= cord_new[2]:
        cord_new[0] = cord_new[2] - 1
    cord_new[1] = np.minimum(223, cord_new[2])
    cord_new[3] = np.minimum(223, cord_new[3])
    if cord_new[1] >= cord_new[3]:
        cord_new[1] = cord_new[3] - 1
    
    if(cord_new[2]-cord_new[0]<=50):
        if(cord_new[2]<223-(50-cord_new[2]+cord_new[0])):
            cord_new[2] += (50-cord_new[2]+cord_new[0])
        else:
            cord_new[2] -= (50-cord_new[2]+cord_new[0])
            
    if(cord_new[3]-cord_new[1]<=50):
        if(cord_new[3]<223-(50-cord_new[3]+cord_new[1])):
            cord_new[3] += (50-cord_new[3]+cord_new[1])
        else:
            cord_new[3] -= (50-cord_new[3]+cord_new[1])

            

    
    if(cord_new[2]-cord_new[0]<=50):
        if(cord_new[2]+40<223):
            cord_new[2] += 40
        else:
            cord_new[0] -= 40
            
    if(cord_new[3]-cord_new[1]<=50):
        if(cord_new[3]<+40<223):
            cord_new[3] += 40
        else:
            cord_new[1] -= 40
    
    cord_new[0] = np.maximum(0, cord_new[0])
    cord_new[1] = np.maximum(0, cord_new[1])
    cord_new[2] = np.maximum(0, cord_new[2])
    cord_new[3] = np.maximum(0, cord_new[3])
    Img_cut = image_Tensor[:, cord_new[1]:cord_new[3], cord_new[0]:cord_new[2]]
#     plt.figure()
#     plt.imshow(Img_cut.cpu().numpy().transpose(1, 2, 0))
#     plt.show()
#    print([cord_new[0],cord_new[1], cord_new[2],cord_new[3]])
    Img_cut=Img_cut.unsqueeze(0)
    Img_new=F.interpolate(Img_cut, size=(224,224), mode='bilinear')

    return Img_new, cord_new, action_type

In [ ]:
# net = resnet50(pretrained=True).cuda()
# from PIL import Image
# import os
# from torchvision import transforms
# results=[]
# preprocess = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])
# for file in os.listdir("../input/testyolonetwork/test/JPEGImages/"):
#     input_image = Image.open("../input/testyolonetwork/test/JPEGImages/"+file)
#     print(img)

#     input_tensor = preprocess(input_image)
#     input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

#     # move the input and model to GPU for speed if available
#     if torch.cuda.is_available():
#         input_batch = input_batch.to('cuda')
#         model.to('cuda')

#     with torch.no_grad():
#         print(input_batch.shape)
#         confidence,features=net(input_batch)
#         print(confidence.shape)
#         print(confidence)
#         print(features.shape)
#         output = model(input_batch)
#     # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
#     #print(output[0])
#     # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
#     probs=torch.nn.functional.softmax(output[0], dim=0)
#     values, indices = torch.max(probs, 0)
#     print("here")
#     print(values)
#     results.append(indices)
# resultsfinal=np.array(results)
# print(np.unique(resultsfinal))

In [ ]:
def compute_iou (box1, box2):                                                                                                                                                             
        '''Compute the intersection over union of two set of boxes, each box is [x1,y1,x2,y2].
        Args:
          box1: (tensor) bounding boxes, sized [N,4].
          box2: (tensor) bounding boxes, sized [M,4].
        Return:
          (tensor) iou, sized [N,M].
        '''

        N = box1.size(0)
        M = box2.size(0)
 
        lt = torch.max(
            box1[:,:2].unsqueeze(1).expand(N,M,2),  # [N,2] -> [N,1,2] -> [N,M,2]
            box2[:,:2].unsqueeze(0).expand(N,M,2),  # [M,2] -> [1,M,2] -> [N,M,2]
        )   
 
        rb = torch.min(
            box1[:,2:].unsqueeze(1).expand(N,M,2),  # [N,2] -> [N,1,2] -> [N,M,2]
            box2[:,2:].unsqueeze(0).expand(N,M,2),  # [M,2] -> [1,M,2] -> [N,M,2]
        )   
 
        wh = rb - lt  # [N,M,2]
        wh[wh<0] = 0  # clip at 0
        inter = wh[:,:,0] * wh[:,:,1]  # [N,M]
 
        area1 = (box1[:,2]-box1[:,0]) * (box1[:,3]-box1[:,1])  # [N,]
        area2 = (box2[:,2]-box2[:,0]) * (box2[:,3]-box2[:,1])  # [M,]
        area1 = area1.unsqueeze(1).expand_as(inter)  # [N,] -> [N,1] -> [N,M]
        area2 = area2.unsqueeze(0).expand_as(inter)  # [M,] -> [1,M] -> [N,M]
 
        iou = inter / (area1 + area2 - inter)
        return iou

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import random

# parameter
gamma = 0.1
epsilon_min = 0.1 # minimum exploration value (the smaller the more exploitation)
epsilon_max = 1.0 # maximum exploration value (the larger the more exploration)
epsilon = epsilon_max
epsilon_dec_steps = 5
epsilon_dec = (epsilon_max - epsilon_min) / epsilon_dec_steps
target_update_interval = 50
max_step = 500
experience = [] # experience replay
experience_buffer_size = 2000
experience_sample_size = 15

# class for defining a state
class State:
    def __init__(self, input_tensor):
        self.input_tensor = input_tensor

    def state_feature(self):
        return self.input_tensor


# Q_learning model
def DQL_model(input=1024, output=6):
    """
    Initialize and return a simple discriminator model.
    """
    model = torch.nn.Sequential(torch.nn.Linear(in_features=input, out_features=1024),
                                torch.nn.ReLU(),
                                torch.nn.Linear(in_features=1024, out_features=output)
            )
    return model


# calculate q_values of given state input
def get_q_values(state, model):
    input_features = state.cuda()
    q_values = model(input_features)
    #q_values = predict_q_values.detach().numpy()
    return q_values


# select action base on q_values of current state
# there are 9 possible action indexed from 0 -> 8
def select_action(epsilon, action_values):
    # do exploitation if epsilon less than random in range (0,1)
    if random.random() > epsilon:
        action = action_values.max(0)[1]

    # do exploration if epsilon greater than random in range (0,1)
    # generate random action index between 0 and 8
    else:
        action = random.randint(0,9)

    return action


# place holder for take_action function need Sina and Arhm function for take_action
def take_action(image_tensor,bbox, action):
    #using the initial_bbox to transform the image using a function
    ############
    # reward is the confidence score
    next_image,next_bbox,action_type= Action(image_tensor,bbox,action,20) #Ahm function return
    
    return next_image,next_bbox,action_type
##########################################################


# compute next state in target network
def compute_target(r, next_state, target_model):
    return r + gamma * (get_q_values(next_state.state_feature(), target_model)).max(0)[1]


# using experience inside experience_replay to train the model
def apply_experience(experience, Qmodel, target_model,flag):
    random.shuffle(experience)
    if len(experience) < experience_sample_size:
         sample_size = len(experience)
    else:
         sample_size = experience_sample_size
    i=0
    for random_exp in experience[0:sample_size]:
        current_state, action_values, action, state_reward, next_state, stopping_condition = random_exp
        # compute q_values of next state at target model
        target_value = compute_target(state_reward, next_state, target_model)
        if (stopping_condition != True):
            # Compute and print loss
            loss = criterion(torch.max(action_values[0]), torch.max(target_value))
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            if(flag==False and i<=sample_size):
                loss.backward(retain_graph=True)
                optimizer.step()
                break
            else:
                loss.backward(retain_graph=True)
                optimizer.step()
            i=i+1


#update the target_model with weight from Qmodel
def update_target_model(Qmodel, target_model):
    target_model.load_state_dict(Qmodel.state_dict())


# initiate model, loss function and optimizer
Qmodel =  DQL_model().cuda()
target_model = DQL_model().cuda()
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(Qmodel.parameters(), lr=1e-4)


# training function for deep Q network, this return the current epsilon to keep track
def DQL_train (ground_truth,init_input_image,state_features, Qmodel, target_model, box_coor,current_epoch, epsilon=epsilon_max):

    # coordinate of the box in list format [x1,y1,x2,y2]
    initial_bbox = box_coor
    bbox = initial_bbox
    # using the initial_bbox to transform the image using a function
    # Sina function
    ############
    step = 0
    total_step = 0
    # create initial state base on parsing state_features
    current_state = State(state_features)
    stopping_condition = False
    ious=[]
    bboxes=[]
    # keep exploring the original image until either max_step is reached or the object has been identified
    while (step < max_step or stopping_condition != True):
        # compute q values and decide the action base on q values of the current state
        action_values = get_q_values(current_state.state_feature().cuda(), Qmodel)
        action = select_action(epsilon, action_values)
        # apply action to the current state to get the next state and reward and check for stopping condition and epoch#
        next_image, next_bbox,action_type= take_action(init_input_image,bbox, action)
        box1=next_bbox.unsqueeze(0)
        box2=ground_truth.unsqueeze(0)
        boundary_box_pred=box1
        ready_box_pred=torch.zeros([1, 4], dtype=torch.float)
        ready_box_pred[:,:2]=boundary_box_pred[:,:2]/14 - 0.5*boundary_box_pred[:,2:4]
        ready_box_pred[:,2:4]=boundary_box_pred[:,:2]/14 + 0.5*boundary_box_pred[:,2:4]

        boundary_box_target=box2
        ready_box_target=torch.zeros([1,4], dtype=torch.float)
        ready_box_target[:,:2]=boundary_box_target[:,:2]/14-0.5*boundary_box_target[:,2:4]
        ready_box_target[:,2:4]=boundary_box_target[:,:2]/14+0.5*boundary_box_target[:,2:4]
        ious.append(compute_iou(ready_box_pred,ready_box_target))
        with torch.no_grad():
            confidence,next_state= net(next_image)
        state_reward=0
        if (confidence[5]>0.7):
            state_reward=1
            stopping_condition=True
        else:
            state_reward=-0.5
        next_state=State(next_state)
        step_experience = (current_state, action_values, action, state_reward, next_state, stopping_condition)

        # save the state action sequence into experience replay
        experience.append(step_experience)

        # apply experience replay
        if(step+1<max_step and stopping_condition!=True):
            apply_experience(experience,Qmodel,target_model,True)
        else:
            print("breaking at")
            print(step)
            apply_experience(experience,Qmodel,target_model,False)


        # move to the next state
        current_state = next_state
        bbox = next_bbox
        bboxes.append(bbox)
        step += 1
        if step>max_step or stopping_condition==True:
            break
        total_step += 1
        #print(action_type)
        if(total_step%100==0):
            print(total_step)

        # if reaching the step update number (10) then update target model with weight from target_model
        if total_step % target_update_interval  == 0:
            update_target_model(Qmodel, target_model)

    # reduce exploration parameter as the we becoming more confident in our action
    if current_epoch < epsilon_dec_steps:
        epsilon -= epsilon_dec

    return epsilon,bboxes,torch.FloatTensor(ious),

# testing function for deep Q network after training
def DQL_testing(state_features, box_coor, Qmodel):

    # coordinate of the box in list format [x1,y1,x2,y2]
    initial_bbox = box_coor
    bbox = initial_bbox
    # using the initial_bbox to transform the image using a function
    # Sina function
    ############
    step = 0
    total_step = 0
    # create initial state base on parsing state_features
    current_state = State(state_features)
    stopping_condition = False

    while step != max_step or stopping_condition != True:
        # compute q values and decide the action base on q values of the current state
        action_values = get_q_values(current_state.state_feature(), Qmodel)
        action = select_action(epsilon, action_values)
        # apply action to the current state to get the next state and reward and check for stopping condition and epoch#
        state_reward, next_state, next_bbox, stopping_condition, current_epoch = take_action(bbox, action)

        # move to the next state
        current_state = next_state
        bbox = next_bbox
        step += 1


In [ ]:
net = resnet50(pretrained=True).cuda()
feature_optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
feature_criterion=torch.nn.BCELoss()

In [ ]:
from PIL import Image
import os
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
results=[]
preprocess = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
file = os.listdir("../input/testyolonetwork/test/JPEGImages/")[15]
input_image = Image.open("../input/testyolonetwork/test/JPEGImages/"+file)
print(input_image)
target_scores=torch.zeros([20])
target_scores[5]=1
targets= open("../input/testyolonetwork/test/voc2007.txt","r")
lines =targets.read()
target=lines.split("\n")
ground_truth_str=[]
for line in range(len(target)):
    if file in target[line]:
        ground_truth_str=target[line].split(" ")[1:5]
ground_truth=torch.zeros([4], dtype=torch.long)
ground_truth[0]=(int(ground_truth_str[0])/500*224)
ground_truth[1]=(int(ground_truth_str[1])/354*224)
ground_truth[2]=(int(ground_truth_str[2])/500*224)
ground_truth[3]=(int(ground_truth_str[3])/354*224)
print(ground_truth)
input_tensor = preprocess(input_image)
plt.figure()
plt.imshow(input_tensor.numpy().transpose(1, 2, 0))
plt.show()
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.cuda()
episodes=[]
for i in range(20):
    confidences,state_features=net(input_batch)
    DQN_state_features=state_features.clone().detach()
    DQN_input_batch=input_batch[0].clone().detach()
    epsilon,bboxes,ious=DQL_train(ground_truth,DQN_input_batch,DQN_state_features,Qmodel,target_model,torch.tensor([50,100,0,50]),i)
    value,index=torch.max(ious,0)
    mask = ious[:] > 0.9
    #print("here")
    #print(mask)
    indices=torch.nonzero(mask)
    losses=[]
    torch.cuda.empty_cache()
    print("matches")
    print(len(indices))
    for i in range(len(indices)):
        next_image,next_bbox,action_type= Action(DQN_input_batch,bboxes[indices[i]],1,0)
        new_confidences,features=net(next_image)
        loss=feature_criterion(new_confidences.cpu(),target_scores)
        print("Feature Loss: ")
        print(loss.item())
        feature_optimizer.zero_grad()
        loss.backward()
        feature_optimizer.step()
#     if(value>0.7):
#         episodes.append(bboxes)
#         box_responsible=bboxes[index]
#         next_image,next_bbox,action_type= Action(DQN_input_batch,box_responsible,1,0)
#         new_confidences,features=net(next_image)
#         loss=feature_criterion(new_confidences.cpu(),target_scores)
#         print("Feature Loss: ")
#         print(loss.item())
#         feature_optimizer.zero_grad()
#         loss.backward()
#         feature_optimizer.step()


    





In [ ]:
from PIL import Image, ImageDraw
img_list=[]
for bb in range(len(bboxes)):
    im=input_image.resize((224,224))
    rimg_draw = ImageDraw.Draw(im)
    current_box=bboxes[bb]
    rimg_draw.rectangle((current_box[0], current_box[1], current_box[2], current_box[3]), fill=None, outline=(255, 0, 0))
    rimg_draw.rectangle((ground_truth[0], ground_truth[1], ground_truth[2], ground_truth[3]), fill=None, outline=(0, 225, 0))
    pil2tensor = transforms.ToTensor()
    img_list.append(pil2tensor(im))
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=100, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
print(index)